In [ ]:
# optional, only for Jupyter
%matplotlib notebook

# General libraries
import numpy as np                # to deal with arrays, vectors, matrices...
import matplotlib.pyplot as plt   # to plot the data
import matplotlib.gridspec as gridspec

# Tensorflow
import os
HOME = os.getenv('HOME')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to get rid of the TF compilation warnings
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
# Only because my system-wide config is tuned, you don't need these lines
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 5,3
mpl.rcParams['font.size'] = 12.0

## Get the data from the public MNIST library
Each sample is a $28\times28$ picture of handwritten numbers.  
The dataset will have dimension $N\times28\times28$

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
inp_shape = x_train.shape[1:]

# Normalize pixel values
x_train = x_train/255
x_test  = x_test/255
x_train = tf.cast(x_train, tf.float32)
x_test  = tf.cast(x_test, tf.float32)

# Report dimensions
print(f'Training: {x_train.shape}')
print(f' Testing: {x_test.shape}')

In [ ]:
# choose a random sample
ind = np.random.randint(0,x_train.shape[0])

img = x_train[ind,:,:]
print(img.shape)

# Plot the sample
fig, ax = plt.subplots()
ax.imshow(img)
ax.set_title(y_train[ind])
ax.grid()
plt.show()

In [ ]:
# Build the model
model = models.Sequential()
model.add( Flatten(input_shape=inp_shape) )
model.add( Dense(512, activation='relu') )
model.add( Dense(100, activation='relu') )
model.add( Dense(10, activation='softmax') )

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training
run = True
if run:
    from time import time
    t_old = time()
    print('Training...')
    history = model.fit(x_train, y_train, epochs=10,
                        validation_data = (x_test, y_test),
                        verbose=1)
    print('...Done in %ss'%(time()-t_old))
else:
    model = models.load('mnist_mlp.h5')

In [ ]:
# plot learning curve
err = history.history['loss']
val_err = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plots grid
fig, ax = plt.subplots(figsize=(5,5))
gs = gridspec.GridSpec(2, 1)
ax0 = plt.subplot(gs[0, 0])
ax1 = plt.subplot(gs[1, 0])

# Loss plots
ax0.plot(err, label='Train')
ax0.plot(val_err, label='Test')
ax0.set_ylabel('Loss')

# Accuracy plots
ax1.plot(acc,label='Train')
ax1.plot(val_acc,label='Test')
ax1.set_ylabel('Accuracy')

# General settings
ax0.set_title('Learning curve')
plt.show()

In [ ]:
loss,acc = model.evaluate(x_test, y_test,verbose=0)
print(f'accuracy: {acc*100:.3f}%')

In [ ]:
# Show Results
predictions = model.predict(x_test)

In [ ]:
# 4 Random examples
samples = [np.random.randint(0,x_test.shape[0]) for _ in range(4)]

fig, ax = plt.subplots(figsize=(5,5))
gs = gridspec.GridSpec(2, 2)
ax0 = plt.subplot(gs[0, 0])
ax1 = plt.subplot(gs[0, 1])
ax2 = plt.subplot(gs[1, 0])
ax3 = plt.subplot(gs[1, 1])

axs = [ax0,ax1,ax2,ax3]
for i in range(len(samples)):
    ax = axs[i]
    ind = samples [i]
    img = x_test[ind,:,:]
    label = y_test[ind]
    predicted = np.argmax(predictions[ind])
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('The NN says: %s'%(predicted))
plt.show()

In [ ]:
# model.save('mnist_mlp.h5')